In [ ]:
construct = 'odi'

total = {'odi' : 9}

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# params
from utils import read_config

config = read_config()

DATAROOT = "../data/"
PLOTROOT = "../plots/"

pros_cons = ['pros', 'cons']

# params
sent_based = True
text = config['TEXTS'][construct]
num_goals = len(config['COUNTS'][construct])
THRESHOLD = config['THRESHOLDS'][construct]
PRESET = config['PRESETS'][construct]
primary_goal_shorthand = config['SHORTHANDS'][construct]
final_goals = config['FINAL'][construct]
scoring = config['SCORING'][construct]
reference_text = config['REFERENCES'][construct]
filename = config['REFERENCE_DATAS'][construct]
sim_num = 0

In [ ]:
from utils import subset_by_percentile, subset_by_percentile_or_preset, sample_data

In [ ]:
main_data = pd.read_csv(DATAROOT + "review_us_master.csv")
onek_data = sample_data(main_data)

In [ ]:
reference = pd.read_csv(DATAROOT + filename, sep = "\t")
reference_name = 'item text'

In [ ]:
main_data = main_data.dropna(subset = [text], axis = 0)
#main_data['embedding'] = list(embeddings)

In [ ]:
from utils import find_similarity, plot_sim_dist

In [ ]:
#### do once and save
# for i in range(num_goals):
#     main_data['%d_sim_1' %(i)] = find_similarity(main_data['embedding'].values,
#                                                sustainability['embedding_1'].values[i])
#     main_data['%d_sim_2' %(i)] = find_similarity(main_data['embedding'].values,
#                                                sustainability['embedding_2'].values[i])  
    
# # save similar data but drop the embeddings
# main_data = main_data.drop('embedding', axis = 1)

# main_data.to_csv(DATAROOT+"intermediate/%s_embedded_sim_review.csv" %(text), sep = "\t", index = False)

In [ ]:
main_datas = {}
main_datas_ = {}
for text in [text]:
    if sent_based:
        embed_file = "intermediate/%s_%s_sent_embedded.csv" %(construct, text)
    else:
        embed_file = "intermediate/%s_embedded_sim_review.csv" %(text) # original full pro or con
        
    main_datas_[text] = pd.read_csv(DATAROOT+embed_file, sep = "\t")
    main_datas[text] = []
    main_datas[text].append(subset_by_percentile_or_preset(main_datas_[text],
                                                           THRESHOLD, simfield = '_sim_1',
                                                           preset = PRESET, num = total[construct]))
    #main_datas[text].append(subset_by_percentile(main_datas_[text], 0.90, simfield = '_sim_2'))

In [ ]:
embed_file

In [ ]:
# TF-IDF validation

In [ ]:
from collections import Counter
from nltk import ngrams
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import string

from string import digits
import re

remove_digits = str.maketrans('', '', digits)
stops = stopwords.words('english')
lemma = nltk.wordnet.WordNetLemmatizer()

def find_keywords(main_datas, fulltext = True, textfield = "whole_review_text",
                  n_gram_range = range(4), sim_num = 0, num_goals = num_goals):
    # step 1: find keywords
    goal_keywords = []
    documents = []
    if sent_based:
        textfield_ = textfield + "_sent"

    print(num_goals)
    # basic preprocessing --- a) lower case, b) remove punctuaction
    for num in range(num_goals):
        dataset = main_datas[textfield][sim_num][num].copy()
        dataset = dataset.dropna(subset = [textfield_], axis = 0)
        #print(len(dataset))
        doc = ' '.join(dataset[textfield].values).lower().translate(string.punctuation).translate(remove_digits)
        doc = re.sub(r'[^\w\s]','',doc).replace('_', '')
        split_doc = [lemma.lemmatize(i, pos="v") for i in doc.split() if i not in stops]
        documents.append(' '.join(split_doc)  )
        
        keywords = []    
        if not fulltext:
            for n in n_gram_range:
                ngram_counts = Counter(ngrams(split_doc, n)).most_common(100)
                keywords.append([" ".join(list(i)) for (i,j) in ngram_counts])
            goal_keywords.append(keywords)          
    
    if fulltext:
        megadoc = [i.split(" ") for i in documents]
        megadoc = [item for sublist in megadoc for item in sublist]
        for n in n_gram_range:
            ngram_counts = Counter(ngrams(megadoc, n)).most_common(100)
            #print(n, ngram_counts[:5])
            keywords.append([" ".join(list(i)) for (i,j) in ngram_counts])
        goal_keywords.append(keywords)
    return goal_keywords, documents

In [ ]:
def get_tfidf(documents, n_gram_range):
    vect = TfidfVectorizer(ngram_range = n_gram_range)
    tfidf_matrix = vect.fit_transform(documents)
    tfidf_scores = pd.DataFrame(tfidf_matrix.toarray(), columns = vect.get_feature_names())
    features = vect.get_feature_names()
    return vect, features, tfidf_scores

In [ ]:
def get_tfidf_scores(keywords, features, tfidf_scores, fulltext = True, num_goals = num_goals):
    
    all_goal_keywords = keywords.copy()

    if not fulltext:
        tfidfs_per_goal = []
        

        for goal in range(num_goals):
            all_goal_keywords[goal] = [item for sublist in all_goal_keywords[goal] for item in sublist]
            #all_goal_keywords[goal] = [" ".join(i) if type(i) == tuple else i for i in all_goal_keywords[goal]]
            shortlist = list(set(all_goal_keywords[goal]).intersection(features))
            tfidfs_per_goal.append(tfidf_scores[shortlist])
        tfidfs = tfidfs_per_goal
    else:
        goal= 0
        all_goal_keywords[goal] = [item for sublist in all_goal_keywords[goal] for item in sublist]
        #all_goal_keywords[goal] = [" ".join(i) if type(i) == tuple else i for i in all_goal_keywords[goal]]
        #print(all_goal_keywords[goal])
        shortlist = list(set(all_goal_keywords[goal]).intersection(features))
        tfidfs = [tfidf_scores[shortlist]]
        
    return tfidfs

In [ ]:
new_order = [0, 1, 7, 8, 2, 3, 5, 6, 12, 4, 9, 10, 11, 13, 14, 15, 16]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()


def get_tfidf_heatmap(tfidfs, goal = 0, title = "all goals TF-IDF",
                      textfield = 'whole_review_text', 
                      mapping = {'whole_review_text' : 'Red'},
                      ordered = False,
                      reduced = False, 
                      new_reduced_order = final_goals,
                      even_more_reduced = False,
                      even_more_reduced_order = [2, 4],
                      reference = reference,
                      reference_name = "Goal"
                     ):
    #f, ax = plt.subplots(figsize=(30, 5))
    
    ordering = range(17)

    df = scaler.fit_transform(tfidfs[goal])
    df = df.sort_values(by=0, ascending=False, axis=1)
    if ordered:
        if reduced:
            if even_more_reduced:
                new_reduced_order = even_more_reduced_order
            df = df.reindex(new_reduced_order)
            ordering = new_reduced_order
        else:
            df = df.reindex(new_order)
            ordering = new_order
        title = title + ' ordered'
    df = df.loc[:, (df < 0.02).any(axis=0)]
    
    
    print(len(df.columns))
    dfs = [df[df.columns[:30]]]
    dfs.append(df[df.columns[30:60]])
    dfs.append(df[df.columns[60:90]])
    dfs.append(df[df.columns[90:120]])
    dfs.append(df[df.columns[120:150]])
    dfs.append(df[df.columns[150:180]])
    dfs.append(df[df.columns[180:210]])
    dfs.append(df[df.columns[210:240]])

    #df = df != 0
    
    fig, ax = plt.subplots(nrows = 4, ncols = 2, figsize = (18, 15))
    
    num = 0
    for i in range(0, 4):
        for j in range(0, 2):
            sns.heatmap(dfs[num], cmap = mapping[textfield], linewidths = .5, ax = ax[i][j])
            labels = list(reference[reference_name])
    
            labels_ = []
            for n in ordering:
                labels_.append(primary_goal_shorthand[labels[n]])

            ax[i][j].set_yticklabels(labels_)
            ax[i][j].yaxis.set_tick_params(rotation=0)
            
            num += 1
            
    #fig.suptitle(title)
    
    plt.tight_layout()
    #plt.savefig(PLOTROOT + "validation/tfidf_sent_based/%s.pdf" %(title))

In [ ]:
primary_goal_shorthand

In [ ]:
new_order = [0, 1, 7, 8, 2, 3, 5, 6, 12, 4, 9, 10, 11, 13, 14, 15, 16]



def get_tfidf_heatmap(tfidfs, goal = 0, title = "all goals TF-IDF",
                      textfield = 'whole_review_text', 
                      mapping = {'whole_review_text' : 'Red'},
                      ordered = False,
                      reduced = False, 
                      new_reduced_order = final_goals,
                      even_more_reduced = False,
                      even_more_reduced_order = [2, 4],
                      reference = reference,
                      reference_name = reference_text,
                      shorthands = primary_goal_shorthand,
                      num_goals = num_goals,
                     ):
    #f, ax = plt.subplots(figsize=(30, 5))
    
    ordering = range(num_goals)

    df = tfidfs[goal]
    df = df.sort_values(by=0, ascending=False, axis=1)
#     if ordered:
#         if reduced:
#             if even_more_reduced:
#                 new_reduced_order = even_more_reduced_order
#             df = df.reindex(new_reduced_order)
#             ordering = new_reduced_order
#         else:
#             df = df.reindex(new_order)
#             ordering = new_order
#         title = title + ' ordered'
    df = df.loc[:, (df < 0.01).any(axis=0)]
    #x_scaled = scaler.fit_transform(df)
    #df = pd.DataFrame(x_scaled, columns = df.columns)
    
    
    print(len(df.columns))
    dfs = [df[df.columns[:50]]]
    dfs.append(df[df.columns[50:100]])
    dfs.append(df[df.columns[100:150]])

    #df = df != 0
    
    fig, ax = plt.subplots(nrows = 3, ncols = 1, figsize = (18, 18))
    
    num = 0
    print(ordering)
    
    for i in range(0, 3):
        for j in range(0, 1):
            #dfs[num] = dfs[num].sort_values(by=num, ascending=False, axis=1)
            sns.heatmap(dfs[num], cmap = mapping[textfield], linewidths = .5, ax = ax[i])
            labels = list(reference[reference_name])
    
            labels_ = []
            for n in ordering:
                labels_.append(shorthands[labels[n]])

            ax[i].set_yticklabels(labels_, fontsize = 18)
            ax[i].yaxis.set_tick_params(rotation=0)
            ax[i].set_xticklabels(dfs[num].columns, fontsize = 16)
            
            num += 1
            
    #fig.suptitle(title)
    
    plt.tight_layout()
    #plt.savefig(PLOTROOT + "validation/tfidf_sent_based/%s.pdf" %(title))

In [ ]:
colormap = {'pros' : 'YlGn', 'cons' : 'OrRd', 'whole_review_text' : 'PuBu'}
  
main_datas = {}
main_datas_ = {}
for text in pros_cons:
    embed_file = "intermediate/%s_%s_sent_embedded.csv" %(construct, text)
    main_datas_[text] = pd.read_csv(DATAROOT+embed_file, sep = "\t")
    main_datas[text] = []
    main_datas[text].append(subset_by_percentile_or_preset(main_datas_[text], THRESHOLD, simfield = '_sim_1',
                                                           preset = PRESET, num = total[construct]))
    #main_datas[text].append(subset_by_percentile(main_datas_[text], 0.90, simfield = '_sim_2'))    

In [ ]:
start = 1
end = 4
n_gram_range = range(start, end)

In [ ]:
for textfield in ["cons"]: 
    per_goal_keywords, documents = find_keywords(main_datas, fulltext = False, textfield = textfield,
                                                 n_gram_range = n_gram_range, sim_num = sim_num,
                                                 num_goals = total[construct])  
    keywords, documents = find_keywords(main_datas, textfield = textfield, n_gram_range = n_gram_range,
                                        sim_num = sim_num, num_goals = total[construct])  
    vect, features, tfidf_scores = get_tfidf(documents, n_gram_range = (start,end-1))
    per_goal_tfidfs = get_tfidf_scores(per_goal_keywords, features, tfidf_scores, 
                                       fulltext = False, num_goals = total[construct])  
    tfidfs = get_tfidf_scores(keywords, features, tfidf_scores)    
    title = "all goals (30 keywords) TF-IDF on %s (Def = %d, \
    Threshold = %0.2f, preset_sim = %0.2f)" %(textfield, sim_num, THRESHOLD, PRESET)    
    title = title + " %d-%d" %(n_gram_range[0], n_gram_range[-1])    
    #get_tfidf_heatmap(tfidfs_, 0, title = title, textfield = textfield, mapping = colormap, ordered = True)  
    get_tfidf_heatmap(tfidfs, 0, title = title, textfield = textfield, mapping = colormap, 
                      ordered = True, reduced = True, reference = reference, reference_name = reference_name,
                      shorthands = primary_goal_shorthand, num_goals = total[construct]) 

In [ ]:
tfidfs[0]

In [ ]:
x = tfidfs[0]
x_scaled = scaler.fit_transform(x)
pd.DataFrame(x_scaled, columns = tfidfs[0].columns)

In [ ]:
x.columns

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

sentence = "He was running and eating at same time. He has bad habit of swimming after playing long hours in the Sun."
punctuations="?:!.,;"
sentence_words = nltk.word_tokenize(sentence)
for word in sentence_words:
    if word in punctuations:
        sentence_words.remove(word)

sentence_words
print("{0:20}{1:20}".format("Word","Lemma"))

for word in sentence_words:
    print ("{0:20}{1:20}".format(word,wordnet_lemmatizer.lemmatize(word, pos="v")))


In [ ]:
def get_handpicked_heatmap(tfidfs, keywords, goal = 0, title = "all goals TF-IDF",
                      textfield = 'whole_review_text', 
                      mapping = {'whole_review_text' : 'Red'},
                      ordered = False,
                      reduced = False, 
                      new_reduced_order = final_goals,
                      even_more_reduced = False,
                      even_more_reduced_order = [2, 4],
                      reference = reference,
                      reference_name = "Goal"
                     ):
    #f, ax = plt.subplots(figsize=(30, 5))
    
    ordering = range(num_goals)

    df = tfidfs[goal]
    df = df.sort_values(by=0, ascending=False, axis=1)
    if ordered:
        if reduced:
            if even_more_reduced:
                new_reduced_order = even_more_reduced_order
            df = df.reindex(new_reduced_order)
            ordering = new_reduced_order
        else:
            df = df.reindex(new_order)
            ordering = new_order
        title = title + ' ordered'
    df = df.loc[:, (df < 0.02).any(axis=0)]
    #x_scaled = scaler.fit_transform(df)
    #df = pd.DataFrame(x_scaled, columns = df.columns)
    
    
    print(len(df.columns))
    dfs = [df[df.columns[:50]]]
    dfs.append(df[df.columns[50:100]])
    dfs.append(df[df.columns[100:150]])

    #df = df != 0
    
    fig, ax = plt.subplots(nrows = 3, ncols = 1, figsize = (18, 18))
    
    num = 0
    for i in range(0, 3):
        for j in range(0, 1):
            #dfs[num] = dfs[num].sort_values(by=num, ascending=False, axis=1)
            sns.heatmap(dfs[num], cmap = mapping[textfield], linewidths = .5, ax = ax[i])
            labels = list(reference[reference_name])
    
            labels_ = []
            for n in ordering:
                labels_.append(primary_goal_shorthand[labels[n]])

            ax[i].set_yticklabels(labels_, fontsize = 18)
            ax[i].yaxis.set_tick_params(rotation=0)
            ax[i].set_xticklabels(dfs[num].columns, fontsize = 16)
            
            num += 1
            
    #fig.suptitle(title)
    
    plt.tight_layout()
    #plt.savefig(PLOTROOT + "validation/tfidf_sent_based/%s.pdf" %(title))

In [ ]:
chosen_keywords = [
  'work', 'great', 'good', 'benefit', 'balance', 'environment', 'people', 'life', 'opportunities', 'pay', 'culture', 'company', 'learn', 'growth', 'lot', 'employees', 'train', 'worklife', 'career', 'job', 'management', 'opportunity', 'time', 'team', 'excellent', 'friendly', 'flexible', 'get', 'development', 'place', 'grow', 'employee', 'hours', 'salary', 'help', 'coworkers', 'advancement', 'care', 'decent', 'support', 'make', 'nice', 'many', 'health', 'leadership', 'experience', 'well', 'diversity', 'compensation', 'new', 'strong', 'provide', 'value', 'schedule', 'flexibility', 'best', 'program', 'amaze', 'really', 'project', 'competitive', 'encourage', 'supportive', 'smart', 'k', 'also', 'take', 'business', 'fun', 'focus', 'home', 'professional', 'community', 'family', 'always', 'within', 'like', 'challenge', 'free', 'give', 'different', 'build', 'skills', 'offer', 'resources', 'technology', 'industry', 'managers', 'everyone', 'perk', 'interest', 'hard', 'atmosphere', 'one', 'level', 'easy', 'positive', 'promote', 'want', 'room'], ['work life', 'life balance', 'great benefit', 'good work', 'worklife balance', 'work environment', 'good benefit', 'benefit good', 'great work', 'balance good', 'benefit great', 'great people', 'good pay', 'balance great', 'pay benefit', 'environment good', 'work culture', 'environment great', 'people work', 'great culture', 'good people', 'benefit work', 'place work', 'great pay', 'good worklife', 'flexible work', 'people great', 'pay good', 'great place', 'work great', 'work home', 'great company', 'people good', 'work good', 'good environment', 'balance work', 'good culture', 'culture good', 'pay great', 'growth opportunities', 'smart people', 'great environment', 'culture great', 'career growth', 'environment work', 'excellent benefit', 'friendly environment', 'job security', 'learn opportunities', 'company work', 'great worklife', 'decent pay', 'benefit pay', 'health benefit', 'lot opportunities', 'flexible schedule', 'opportunities great', 'care employees', 'work hours']

In [ ]:
def get_rotated_heatmap(tfidfs, goal = 0, title = "all goals TF-IDF",
                      textfield = 'whole_review_text', 
                      mapping = {'whole_review_text' : 'Red'},
                      ordered = False,
                      reduced = False, 
                      new_reduced_order = final_goals,
                      even_more_reduced = False,
                      even_more_reduced_order = [2, 4],
                      chosen_keywords = None,
                      reference = reference,
                      reference_name = reference_text
                     ):
    #f, ax = plt.subplots(figsize=(30, 5))
    
    ordering = range(num_goals)

    df = tfidfs[goal]
    if chosen_keywords:
        df = df[chosen_keywords]
    df = df.sort_values(by=0, ascending=False, axis=1)
    if ordered:
        if reduced:
            if even_more_reduced:
                new_reduced_order = even_more_reduced_order
            df = df.reindex(new_reduced_order)
            ordering = new_reduced_order
        else:
            df = df.reindex(new_order)
            ordering = new_order
        title = title + ' ordered'
    df = df.loc[:, (df < 0.02).any(axis=0)]
    x_scaled = scaler.fit_transform(df)
    df = pd.DataFrame(x_scaled, columns = df.columns)
    
    
    print(len(df.columns))
    #print(list(df.columns))
#     print(keywords)
#     df = df[chosen_keywords]
#     print(df)
    dfs = [df[df.columns[:25]]]
    dfs.append(df[df.columns[30:60]])
    dfs.append(df[df.columns[60:90]])
    

    #df = df != 0
    
    fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (16, 5))
    
    num = 0
    for i in range(0, 1):
        for j in range(0, 1):
            #dfs[num] = dfs[num].sort_values(by=num, ascending=False, axis=1)
            sns.heatmap(dfs[num], cmap = mapping[textfield], linewidths = .5, ax = ax)
            labels = list(reference[reference_name])
    
            labels_ = []
            for n in ordering:
                labels_.append(primary_goal_shorthand[labels[n]])

            ax.set_yticklabels(labels_, fontsize = 18)
            ax.yaxis.set_tick_params(rotation=0)
            ax.set_xticklabels(dfs[num].columns, fontsize = 14)
            
            bottom, top = ax.get_ylim()
            ax.set_ylim(bottom + 0.5, top - 0.5)
            
            num += 1
            
    #fig.suptitle(title)
    
    plt.tight_layout()
    plt.savefig(PLOTROOT + 'tfidf.pdf')
    return df